In [19]:
from utils.constants import UNIVARIATE_ARCHIVE_NAMES as ARCHIVE_NAMES

from utils.utils import read_all_datasets
from utils.utils import transform_labels
from utils.utils import create_directory
from utils.utils import run_length_xps
from utils.utils import generate_results_csv
from data_formatting import split_sequence_overlap, split_sequence_nooverlap, split_sequence, split_train_test, normalize_data, set_targets
import parameters
parameters.initialize_parameters()


import utils
import numpy as np
import pandas as pd
import sys
import sklearn
import torch
from torch.utils.data import TensorDataset, DataLoader
import tensorflow as tf

In [20]:
#Parameters
option = 'InceptionTime'
#list_targets = [1, 2]
list_labels = [0, 1]
root_dir = '../inception_time_root_dir'
xps = ['use_bottleneck', 'use_residual', 'nb_filters', 'depth',
       'kernel_size', 'batch_size']

csv_file = "../inception_time_root_dir/archives/All_Subs_Diff_Modules_nofilter_withoutAUc.csv"
train_df = pd.read_csv(csv_file,  delimiter=",")  # 101 features (only AU_r)

## Functions

In [21]:
def fit_classifier(x_train, y_train, x_test, y_test, y_true, nb_classes, classifier_name, output_directory):
    input_shape = x_train.shape[1:]

    classifier = create_classifier(classifier_name, input_shape, nb_classes,
                                   output_directory)

    classifier.fit(x_train, y_train, x_test, y_test, y_true)


def create_classifier(classifier_name, input_shape, nb_classes, output_directory,
                      verbose=False, build=True):
    print("Creating Classifier\n")
    if classifier_name == 'nne':
        from classifiers import nne
        return nne.Classifier_NNE(output_directory, input_shape,
                                  nb_classes, verbose)
    if classifier_name == 'inception':
        from classifiers import inception
        return inception.Classifier_INCEPTION(output_directory, input_shape, nb_classes, verbose,
                                              build=build)


def get_xp_val(xp):
    if xp == 'batch_size':
        xp_arr = [16, 32, 128]
    elif xp == 'use_bottleneck':
        xp_arr = [False]
    elif xp == 'use_residual':
        xp_arr = [False]
    elif xp == 'nb_filters':
        xp_arr = [16, 64]
    elif xp == 'depth':
        xp_arr = [3, 9]
    elif xp == 'kernel_size':
        xp_arr = [8, 64]
    else:
        raise Exception('wrong argument')
    return xp_arr

def split_x_y(data_loader):
  x = []
  y = []
  for i, (x_batch, y_batch) in enumerate(data_loader):
    x.append(x_batch)
    y.append(y_batch)
  x = torch.cat(x, dim=0)
  y = torch.cat(y, dim=0)
  return x, y

# Training on one specific subject
def prepare_data(list_targets, list_labels, test_subj, train_df):
    # Select only the classes we want to predict
    train_df, nclasses, targets_numpy = set_targets(train_df, list_targets, list_labels)

    # Convert the subject names (strings) into numbers
    subjects = pd.factorize(train_df['Subject'])[0]

    # Normalise the features
    features_numpy = normalize_data(train_df, False) #parameters.normalise_individual_subjects
    del train_df

    subj = np.unique(subjects)

    test_subj -= 1

    test_idx = np.array([subj[test_subj]])

    train_idx = np.setxor1d(subj, test_idx)

    print("Generating train/val/test split...")
    features_train, targets_train, features_test, targets_test = split_train_test(targets_numpy, features_numpy, subjects, train_idx, test_idx)

    print("Generating sequences...")
    features_train, targets_train = split_sequence_overlap(features_train, targets_train, parameters.seq_dim, parameters.overlap_size)
    
    # Overlap or no
    if parameters.test_with_subsequences:
      features_test, targets_test = split_sequence_overlap(features_test, targets_test, parameters.test_seq_dim, parameters.test_overlap_size)
    else:
      features_test, targets_test = split_sequence_nooverlap(features_test, targets_test, parameters.test_seq_dim, parameters.test_overlap_size)


    # Create feature and targets tensor for train set. We need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
    featuresTrain = torch.from_numpy(features_train)
    targetsTrain = torch.from_numpy(targets_train).type(torch.LongTensor)  # data type is long


    # Pytorch train and validation sets
    train = TensorDataset(featuresTrain, targetsTrain)
    
    # Data loader
    train_loader = DataLoader(train, batch_size=parameters.batch_size, shuffle=True)

    # Create feature and targets tensor for test set
    if parameters.test_with_subsequences:
      featuresTest = torch.from_numpy(features_test)
      targetsTest = torch.from_numpy(targets_test).type(torch.LongTensor)  # data type is long
      test = TensorDataset(featuresTest, targetsTest)
      test_loader = DataLoader(test, batch_size=parameters.batch_size, shuffle=False)

    # Split test and train sets
    x_test, y_test = split_x_y(test_loader)
    x_train, y_train = split_x_y(train_loader)

    # save orignal y because later we will use binary
    y_true = y_test.to(torch.int64)
    y_true_train = y_train.to(torch.int64)
    # transform the labels from integers to one hot vectors
    enc = sklearn.preprocessing.OneHotEncoder()
    enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
    y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
    y_test = enc.transform(y_test.reshape(-1, 1)).toarray()

    if len(x_train.shape) == 2:  # if univariate
        print("Univariate time series classification\n")
        # add a dimension to make it multivariate with one dimension
        x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
        x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

    nb_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))
    x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
    y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)
    x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)
    y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
    y_true_train = tf.convert_to_tensor(y_true_train, dtype=tf.float32)
    
    return x_train, y_train, x_test, y_test, y_true, nb_classes, y_true_train, enc

def inception_time(list_targets, test_subj):
    # run nb_iter_ iterations of Inception on the whole TSC archive
    classifier_name = 'inception'
    archive_name = ARCHIVE_NAMES[0]
    model_couple = str(list_targets[0]) + '_' + str(list_targets[1]) + '_' + str(test_subj)
    nb_iter_ = 5

    print(f'Running InceptionTime on {list_targets[0]}_{list_targets[1]} with test subject {test_subj}')

    for iter in range(nb_iter_):
        print('\titeration', iter)

        trr = ''
        if iter != 0:
            trr = '_itr_' + str(iter)

        tmp_output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + trr + '/'

        output_directory = tmp_output_directory + model_couple + '/'

        temp_output_directory = create_directory(output_directory)

        if temp_output_directory is None:
            print('\t\tAlready_done', tmp_output_directory, model_couple)
            continue

        x_train, y_train, x_test, y_test, y_true, nb_classes, y_true_train, enc = prepare_data(list_targets, list_labels, test_subj, train_df)

        fit_classifier(x_train, y_train, x_test, y_test, y_true, nb_classes, classifier_name, output_directory)

        print(f'\t\titeration {iter} DONE')

        # the creation of this directory means
        create_directory(output_directory + '/DONE')

    print(f'\t\Running InceptionTime on the whole TSC archive for {model_couple}')
    # run the ensembling of these iterations of Inception
    classifier_name = 'nne'

    tmp_output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + '/'

    x_train, y_train, x_test, y_test, y_true, nb_classes, y_true_train, enc = prepare_data(list_targets, list_labels, test_subj, train_df)

    output_directory = tmp_output_directory + model_couple + '/'

    fit_classifier(x_train, y_train, x_test, y_test, y_true, nb_classes, classifier_name, output_directory)

    print('\t\t\t\tDONE')


## Training

In [ ]:
for i in range(2):
    for j in range(i+1, 2):
        inception_time([i, j], 3)

## Old main

In [8]:
############################################### OLD MAIN
if option == 'InceptionTime':
    # run nb_iter_ iterations of Inception on the whole TSC archive
    classifier_name = 'inception'
    archive_name = ARCHIVE_NAMES[0]
    dataset_name = utils.constants.dataset_names_for_archive[archive_name][0]
    nb_iter_ = 5

    for iter in range(nb_iter_):
        print('\t\titer', iter)

        trr = ''
        if iter != 0:
            trr = '_itr_' + str(iter)

        tmp_output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + trr + '/'

        print('\t\t\tdataset_name: ', dataset_name)

        x_train, y_train, x_test, y_test, y_true, nb_classes, y_true_train, enc = prepare_data(list_targets, list_labels, 3, train_df)

        output_directory = tmp_output_directory + dataset_name + '/'

        temp_output_directory = create_directory(output_directory)

        if temp_output_directory is None:
            print('Already_done', tmp_output_directory, dataset_name)
            continue

        fit_classifier(x_train, y_train, x_test, y_test, y_true, nb_classes, classifier_name, output_directory)

        print(f'\t\t\t\tIteration {iter} DONE')

        # the creation of this directory means
        create_directory(output_directory + '/DONE')

    print('\t\Running InceptionTime on the whole TSC archive')
    # run the ensembling of these iterations of Inception
    classifier_name = 'nne'


    tmp_output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + '/'

    x_train, y_train, x_test, y_test, y_true, nb_classes, y_true_train, enc = prepare_data(list_targets, list_labels, 1, train_df)

    output_directory = tmp_output_directory + dataset_name + '/'

    fit_classifier(x_train, y_train, x_test, y_test, y_true, nb_classes, classifier_name, output_directory)

    print('\t\t\t\tDONE')

elif option == 'generate_results_csv':
    clfs = []
    itr = '-0-1-2-3-4-'
    inceptionTime = 'nne/inception'
    # add InceptionTime: an ensemble of 5 Inception networks
    clfs.append(inceptionTime + itr)
    # add InceptionTime for each hyperparameter study
    for xp in xps:
        xp_arr = get_xp_val(xp)
        for xp_val in xp_arr:
            clfs.append(inceptionTime + '/' + xp + '/' + str(xp_val) + itr)
    df = generate_results_csv('results.csv', root_dir, clfs)
    print(df)


		iter 0
			dataset_name:  All_Subs_Diff_Modules_nofilter_withoutAUc
Generating train/val/test split...
Generating sequences...
Already_done ../inception_time_root_dir/results/inception/TSC/ All_Subs_Diff_Modules_nofilter_withoutAUc
		iter 1
			dataset_name:  All_Subs_Diff_Modules_nofilter_withoutAUc
Generating train/val/test split...
Generating sequences...
Creating Classifier

Fitting the model



c:\Users\Nathan\Documents\5IF\PSAT\Analyse-de-conscience\env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Saving last model

Predicting

12/12 [==============================] - 7s 517ms/step
Saving predictions

saving Y_pred

convert predicted from binary to integer

df_metrics.csv saved

1

2

3: df_best_model

4: df_best_model -> train loss

5: df_best_model -> train acc

6: df_best_model -> number of epochs

7: save df_best_model csv 

saving df_metrics

clearing session, next instruction is return

				DONE
		iter 2
			dataset_name:  All_Subs_Diff_Modules_nofilter_withoutAUc
Generating train/val/test split...
Generating sequences...
Creating Classifier

Fitting the model



c:\Users\Nathan\Documents\5IF\PSAT\Analyse-de-conscience\env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Saving last model

Predicting

12/12 [==============================] - 7s 515ms/step
Saving predictions

saving Y_pred

convert predicted from binary to integer

df_metrics.csv saved

1

2

3: df_best_model

4: df_best_model -> train loss

5: df_best_model -> train acc

6: df_best_model -> number of epochs

7: save df_best_model csv 

saving df_metrics

clearing session, next instruction is return

				DONE
		iter 3
			dataset_name:  All_Subs_Diff_Modules_nofilter_withoutAUc
Generating train/val/test split...
Generating sequences...
Creating Classifier

Fitting the model



c:\Users\Nathan\Documents\5IF\PSAT\Analyse-de-conscience\env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Saving last model

Predicting

12/12 [==============================] - 9s 665ms/step
Saving predictions

saving Y_pred

convert predicted from binary to integer

df_metrics.csv saved

1

2

3: df_best_model

4: df_best_model -> train loss

5: df_best_model -> train acc

6: df_best_model -> number of epochs

7: save df_best_model csv 

saving df_metrics

clearing session, next instruction is return

				DONE
		iter 4
			dataset_name:  All_Subs_Diff_Modules_nofilter_withoutAUc
Generating train/val/test split...
Generating sequences...
Creating Classifier

Fitting the model



c:\Users\Nathan\Documents\5IF\PSAT\Analyse-de-conscience\env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Saving last model

Predicting

12/12 [==============================] - 10s 679ms/step
Saving predictions

saving Y_pred

convert predicted from binary to integer

df_metrics.csv saved

1

2

3: df_best_model

4: df_best_model -> train loss

5: df_best_model -> train acc

6: df_best_model -> number of epochs

7: save df_best_model csv 

saving df_metrics

clearing session, next instruction is return

				DONE
	\Running InceptionTime on the whole TSC archive
Generating train/val/test split...
Generating sequences...
Creating Classifier

				DONE
